In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from PIL import Image
import cv2
import pandas as pd 
import skimage
from pathlib import Path

In [ ]:
def pltcnt(c, ax=None, **kwargs):
    if ax:
        ax.plot([p[0][0] for p in c], [ p[0][1] for p in c ], **kwargs)
    else:
        plt.plot([p[0][0] for p in c], [ p[0][1] for p in c ],**kwargs)

# Dataset: Digitize-P&ID

Path

In [ ]:
dbpath = Path("./DigitizePID_Dataset")

Masks, images, info

In [ ]:
imagepath = dbpath /  "image_2"
imageformat = "jpg"
maskpath  = dbpath /  "mask"
maskformat = "png"

def im2mask(image):
    return maskpath / f"{image.stem}_mask.{maskformat}"
def mask2im(mask):
    return imagepath / f"{mask.stem}_mask.{imageformat}"
def im2info(image):
    dfs = {  }
    for file in  (dbpath / image.stem).glob("*.npy"):
        data = np.load(str(file), allow_pickle=True)
        name = file.stem.split("_")[-1]
        dfs[name] = pd.DataFrame(data)
    return dfs

In [ ]:
# %matplotlib widget

In [ ]:
%matplotlib tk

def draw_rects(img, tp_br_points, color=(255,0,0), **kwargs):
    for p1, p2 in tp_br_points:
        cv2.rectangle(img, p1, p2, color=color, **kwargs)

# for image in imagepath.glob(f"*.{imageformat}"):

image = imagepath / f"1.{imageformat}"

im = cv2.imread(str(image))
mask = cv2.imread(str(im2mask(image)), cv2.IMREAD_GRAYSCALE)

# 375, 250
# 5630, 4300
roi = (slice(250, 4300), slice(375, 5630))
im = im[roi]
mask = mask[roi]

tr = np.array([375,250])

# im 
contours, hierarchy = cv2.findContours(255*(mask>0).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


data = im2info(image)
draw = im.copy()

# Linetypes
solid_lines = np.stack(data["lines"][data["lines"][3] == 'solid'].iloc[:,1]).reshape(-1,2,2) - tr
dashed_lines = np.stack(data["lines"][data["lines"][3] == 'dashed'].iloc[:,1]).reshape(-1,2,2) - tr
def draw_pipelines(image):
    draw = image.copy()
    draw = cv2.drawContours(draw, solid_lines, -1, (255, 255, 0), thickness=2)
    draw = cv2.drawContours(draw, dashed_lines, -1, (0, 255, 255), thickness=2)
    return draw
# cnts2 = np.stack(data["lines2"].iloc[:,:3]).reshape(-1,2,2)


# 
cnts2 = data["lines2"].iloc[:,:4].to_numpy().reshape(-1,2,2) - tr
draw = cv2.drawContours(draw, cnts2, -1, (0, 255, 255), thickness=2)


def draw_symbols(image):
    draw = image.copy()
    for i, group in data["symbols"].groupby(2):
        color = (np.random.rand(3)*255).astype(np.uint8)
        symbols = np.stack(group.iloc[:,1]).reshape(-1,2,2) - tr
        draw_rects(draw, symbols, color=[int(c) for c in color], thickness=2)
    return draw

text = np.stack(data["words"].iloc[:,1]).reshape(-1,2,2) - tr
def draw_text_boxes(image):
    draw = image.copy()
    draw_rects(draw, text, color=(255,0,255), thickness=1)
    return draw
# draw = cv2.drawContours(draw, text, -1, (255, 0, 0), thickness=2)


# draw = cv2.drawContours(draw, contours, -1, (255, 0, 0), thickness=2)
# draw = cv2.drawContours(draw, symbols, -1, (0, 0, 255), thickness=2)


# im = cv2.imread("test.jpg")
draw = draw_pipelines(draw)
draw = draw_symbols(draw)
draw = draw_text_boxes(draw)
plt.imshow(draw)


## Thresh

In [ ]:
gray = np.mean(im,axis=-1).astype(np.uint8)
t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
plt.imshow(thresh)

## Edges

In [ ]:
dx = cv2.Sobel(gray,cv2.CV_32FC1, 0,1)
dy = cv2.Sobel(gray,cv2.CV_32FC1, 1,0)
grad = np.sqrt(dx**2+dy**2)

fig, axs = plt.subplots(1,2, sharex=True, sharey=True)
t, grad_thresh = cv2.threshold((grad*255/grad.max()).astype(np.uint8), 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
axs[0].imshow(grad_thresh)
axs[1].imshow(grad)

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ksize=(7,7))

grad = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)
plt.imshow(grad)

In [ ]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

skel = skimage.morphology.skeletonize(thresh//255, method="lee")
axs[0].imshow(skel)
axs[1].imshow(thresh)

In [ ]:
contours, hierarchy = cv2.findContours(255-thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

plt.imshow(thresh)
im2 = np.zeros_like(im)

# hierarchy is [next, previous, firstChild, parent]
parents = hierarchy[0,:,-1]
# draw contours with no parents
cv2.drawContours(im2, np.array(contours)[parents==0], -1, color=(255,255,255), thickness=1)
for c in contours:
    pltcnt(c, color="r")

plt.figure()
plt.imshow(im2)

## Hough circles

In [ ]:
hough_input = skel

radii = np.arange(110,125)/2
# radii = np.arange(10,20)
circles = skimage.transform.hough_circle(hough_input, radii, normalize=False)
_, i, j = np.unravel_index(np.argsort(-circles.ravel()),circles.shape)

In [ ]:
accum, cx, cy, rad = skimage.transform.hough_circle_peaks(circles, radii, min_xdistance=np.min(radii).astype(int), min_ydistance=np.min(radii).astype(int))

In [ ]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

axs[0].imshow(im, cmap="gray")
draw = np.zeros_like(im)

# Max diff.
lim = accum[np.argmax(np.abs(np.diff(accum)))]

fit = accum > lim*0.9

for x,y, r, f, a in zip(cx, cy, rad, fit, accum):
    # draw =cv2.circle(draw, (int(x),int(y)), int(r), color=(255,0,0))
# plt.plot(cx[:50],cy[:50],"rx")
    a = a/accum.max()
    axs[0].add_patch(plt.Circle((x,y), r, color=(a if f else 0,0,a), fill=False, linewidth=a*3))

axs[1].imshow(hough_input)

In [ ]:
fig.tight_layout()

In [ ]:
circ_x, circ_y = skimage.draw.circle_perimeter(int(x),int(y),int(r), shape=draw.shape)
plt.imshow(im)
m = np.zeros_like(im)
m[circ_x, circ_y]=(255,0,0)
plt.imshow(m)

In [ ]:
plt.plot(accum)
plt.plot(np.diff(accum))

In [ ]:
accum

# Graph?


In [ ]:
contours, hierarchy = cv2.findContours(skel, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

im2 = np.zeros_like(im)
# hierarchy is [next, previous, firstChild, parent]
# draw contours with no parents
# cv2.drawContours(im2, contours, -1, color=(255,255,255), thickness=1)
plt.imshow(im2)
for c in contours:
    pltcnt(c)

# plt.figure()
# plt.imshow(im2)

## Morphology hats

In [ ]:
def draw_gt(image):
    image = draw_pipelines(image)
    image = draw_symbols(image)
    image = draw_text_boxes(image)
    return image

In [ ]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

gray = np.mean(im,axis=-1).astype(np.uint8)
t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
skel = skimage.morphology.skeletonize(thresh//255, method="lee")

kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(25,25))

blackhat = cv2.morphologyEx(skel, cv2.MORPH_BLACKHAT, kern)
tophat = cv2.morphologyEx(skel, cv2.MORPH_TOPHAT, kern)

draw = draw_symbols(cv2.cvtColor(blackhat*255, cv2.COLOR_GRAY2RGB))

axs[0].imshow(skel)
axs[1].imshow(draw)

In [ ]:
fig.tight_layout()